In [1]:
import pandas as pd
import numpy as np

**read_data_small** is the function to read in the small dataset about 30 MB

In [2]:
def read_data_small():
    X_train = pd.read_csv("data_small/X_train_small.csv")
    X_test = pd.read_csv("data/X_test.csv")
    y_train = np.asarray(pd.read_csv("data_small/y_train_small.csv", header=None)[0])
    return X_train, X_test, y_train

**read_data_big** is the function to read in the big dataset about 100 MB

In [3]:
def read_data_big():
    X_train = pd.read_csv("data_big/X_train_big.csv")
    X_test = pd.read_csv("data/X_test.csv")
    y_train = np.asarray(pd.read_csv("data_big/y_train_big.csv", header=None)[0])
    return X_train, X_test, y_train

**read_data** is the function to read in the whole dataset about 1.5 G

In [4]:
def read_data():
    X_train = pd.read_csv("data/X_train.csv")
    X_test = pd.read_csv("data/X_test.csv")
    y_train = np.asarray(pd.read_csv("data/y_train.csv", header=None)[0])
    return X_train, X_test, y_train

# Insert Your Code Here

**detect_spoofying** is the function for training the classifier and classify the results. 

Here we provide an simple example.

In [5]:
### import libraries here ###
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_validate

### code classifier here ###
def format_data(df):

    # append numberical columns
    rst = df.loc[:,["price","volume","bestBid","bestAsk",'bestBidVolume',
                    'bestAskVolume','lv2Bid', 'lv2BidVolume','lv2Ask', 
                    'lv2AskVolume', 'lv3Bid', 'lv3BidVolume', 'lv3Ask',
                    'lv3AskVolume']]
    
    # encode the binaries
    rst["isBid"] = df.isBid*1
    rst["isBuyer"] = df.isBuyer*1
    rst["isAggressor"] = df.isAggressor*1
    rst["type"] = (df.type == "ORDER")*1
    rst["source"] = (df.source=="USER")*1
    
    # parse the order id data
    rst["orderId"] = df.orderId.str.split('-').str[-1]
    rst["tradeId"] = df.tradeId.str.split('-').str[-1]
    rst["bidOrderId"] = df.bidOrderId.str.split('-').str[-1]
    rst["askOrderId"] = df.askOrderId.str.split('-').str[-1]
    
    # encode the multiple lable data
    tmp_operation = pd.DataFrame(pd.get_dummies(df.operation), columns=df.operation.unique()[:-1])
    rst = pd.concat([rst, tmp_operation], axis=1)
    # tmp_endUserRef = pd.DataFrame(pd.get_dummies(df.endUserRef), columns=df.endUserRef.unique()[:-1])
    # rst = pd.concat([rst, tmp_endUserRef], axis=1)
    
    return rst

def detect_spoofying(X_train, X_test, y_train):
    
    # clean up the data
    X_concat = pd.concat([X_train, X_test])
    X_clean = format_data(X_concat)
    
    # feel free to add more columns inferred from data
    # smartly engineered features can be very useful to improve the classification resutls
    X_clean["timeSinceLastTrade"] = pd.concat([X_train.loc[:,["timestamp","endUserRef"]].groupby("endUserRef").diff(),
                                              X_test.loc[:,["timestamp","endUserRef"]].groupby("endUserRef").diff()])
    
    X_clean = X_clean.fillna(-1)
    X_train_clean = X_clean.iloc[:X_train.shape[0],:]
    X_test_clean = X_clean.iloc[X_train.shape[0]:,:]
    X_train_clean_scaled = scale(X_train_clean)
    X_test_clean_scaled = scale(X_test_clean)

    # fit classifier
    clf = LogisticRegression(random_state=0, class_weight='balanced').fit(X_train_clean_scaled, y_train)
    y_train_prob_pred = clf.predict_proba(X_train_clean_scaled)
    y_test_prob_pred = clf.predict_proba(X_test_clean_scaled)
    
    return y_train_prob_pred, y_test_prob_pred

**score** is the function that we use to compare the results. An example is provided with scoring the predictions for the training dataset. True labels for the testing data set will be supplied to score the predictions for testing dataset.

Score is based on cohen's kappa measurement. https://en.wikipedia.org/wiki/Cohen%27s_kappa

In [6]:
from sklearn.metrics import cohen_kappa_score

def score(y_pred, y_true):
    """
    y_pred: a numpy 4d array of probabilities of point assigned to each label
    y_true: a numpy array of true labels
    """
    y_pred_label = np.argmax(y_pred, axis=1)
    return cohen_kappa_score(y_pred_label, y_true)

### Optional: k-fold cross validation

In [ ]:
### optional: examples of k-fold cross validation ###
# k-fold cross validation can help you compare the classification models
from sklearn.model_selection import KFold
n = 5 # here we choose a 10 fold cross validation
kf = KFold(n_splits = n)
X_train, X_test, y_train = read_data_small()
kf.get_n_splits(X_train)
print(kf)
kf_scores = pd.DataFrame(np.zeros([n,2]), columns=["train score", "test score"])
rowindex = 0
for train_index, test_index in kf.split(X_train):
    print("TRAIN:", train_index, "TEST:", test_index)
    print(X_train.index)
    print(y_train)
    X_train_kf, X_test_kf = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_kf, y_test_kf = y_train[train_index], y_train[test_index]
    y_train_prob_pred_kf, y_test_prob_pred_kf = detect_spoofying(X_train_kf, X_test_kf, y_train_kf)
    score_train_kf = score(y_train_prob_pred_kf, y_train_kf)
    score_test_kf = score(y_test_prob_pred_kf, y_test_kf)
    kf_scores.iloc[rowindex, 0] = score_train_kf
    kf_scores.iloc[rowindex, 1] = score_test_kf
    rowindex += 1

In [ ]:
kf_scores

**wrapper** is the main function to read in unzipped data and output a score for evaluation. In addition, the function returns the y probability matrix (both train and test) for grading. More details about submitting format are outlined below.

In [7]:
def wrapper():
    # read in data
    X_train, X_test, y_train = read_data_small()
    # or if you have the computational power to work with the big data set, 
    # you can comment out the read_data_samll line and uncomment the following read_data_big
    # X_train, X_test, y_train = read_data_big()
    
    # process the data, train classifier and output probability matrix
    y_train_prob_pred, y_test_prob_pred = detect_spoofying(X_train, X_test, y_train)
    
    # score the predictions
    score_train = score(y_train_prob_pred, y_train)
    # score_test = score(y_test_prob_pred, y_test)
    
    # return the scores
    return score_train, y_train_prob_pred, y_test_prob_pred

Call function wrapper:

In [8]:
score_train, y_train_prob_pred, y_test_prob_pred = wrapper()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:50: DataConversionWarning: Data with input dtype uint8, int64, float64, object were all converted to float64 by the scale function.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:51: DataConversionWarning: Data with input dtype uint8, int64, float64, object were all converted to float64 by the scale function.
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Score for training data set is:

In [9]:
score_train

0.126419591062796

### Submission Format

The classifier function wrote should return a 4d nparray with 4 columns. The columns are corresponding to the class labels: 0, 1, 2, 3. Please see examples below.

In [10]:
y_train_prob_pred

array([[7.83097653e-01, 2.44629119e-03, 6.78971413e-03, 2.07666342e-01],
       [9.99980261e-01, 2.14218084e-11, 1.53816170e-06, 1.82008654e-05],
       [9.99980878e-01, 2.38099490e-11, 1.47360641e-06, 1.76482024e-05],
       ...,
       [9.99778296e-01, 6.43165474e-05, 8.41143032e-05, 7.32733779e-05],
       [9.99826615e-01, 1.18900506e-05, 3.98983951e-05, 1.21596640e-04],
       [3.04684576e-01, 2.51657069e-01, 2.53411861e-01, 1.90246495e-01]])

In [11]:
y_test_prob_pred

array([[3.99732835e-01, 2.63298846e-02, 2.56046328e-01, 3.17890953e-01],
       [9.99912789e-01, 1.00006429e-06, 2.18574410e-05, 6.43533056e-05],
       [9.99808886e-01, 1.97358546e-05, 1.16760450e-04, 5.46179063e-05],
       ...,
       [9.99897872e-01, 3.07217446e-07, 1.19738190e-05, 8.98466626e-05],
       [9.99917771e-01, 6.81250331e-07, 1.75267258e-05, 6.40214658e-05],
       [2.45286966e-01, 2.67584009e-01, 2.81561107e-01, 2.05567918e-01]])

In [13]:
y_test_prob_pred.shape

(2565627, 4)

### Write test results to csv files

Please rename your file to indicate which data set you are working with. 

- If you are using the small dataset: *y_train_prob_pred_small.csv* and *y_test_prob_pred_small.csv*
- If you are using the small dataset: *y_train_prob_pred_big.csv* and *y_test_prob_pred_big.csv*
- If you are using the original dataset: *y_train_prob_pred.csv* and *y_test_prob_pred.csv*

In [12]:
pd.DataFrame(y_train_prob_pred).to_csv("y_train_prob_pred.csv")
pd.DataFrame(y_test_prob_pred).to_csv("y_test_prob_pred.csv")